Я решил распарсить все отзывы на смартфоны с сайта DNS

In [1]:
import requests
import bs4
import pickle


In [111]:
%%writefile dns_reviews.py
import requests
import bs4
from multiprocessing import Pool
import codecs
from functools import reduce
import pickle

DNS_WEBPAGE = 'https://www.dns-shop.ru'

def get_all_products(url):
    print('>>> Parsing page '+url)
    text = requests.get(url).text
    parser = bs4.BeautifulSoup(text, 'lxml')
    
    product_links = parser.findAll('div', attrs='title')
    return [DNS_WEBPAGE+product.a['href'] for product in product_links]

PAGES_LIST = [DNS_WEBPAGE + '/catalog/17a8a01d16404e77/smartfony/?p={}'.format(num) for num in range(1,51)]

def get_opinions(product_url):
    print('\t>>> Parsing product '+product_url)
    text = requests.get(product_url+'opinion/').text
    parser = bs4.BeautifulSoup(text, 'lxml')
    
    opinions = parser.findAll('div', attrs='opinion-item')
    
    opnions_w_raiting = []
    for opinion_parser in opinions:
#         print(opinion_parser.prettify())
        rating = opinion_parser.find('div', attrs='product-item-rating rating')['data-rating']
        descriptions = opinion_parser.findAll('span', attrs='description-text')
        opinion_text = ' '.join([descr.text for descr in descriptions])
        opnions_w_raiting.append((opinion_text, int(rating)))
        
    return opnions_w_raiting


if __name__ == '__main__':
    p = Pool(100)

    map_products = p.map(get_all_products, PAGES_LIST)
    reduce_products = reduce(lambda x,y: x + y, map_products)
    
    map_reviews = p.map(get_opinions, reduce_products)
    reduce_reviews = reduce(lambda x,y: x + y, map_reviews)
    
    with open('all_dns.txt', 'wb') as output_file:
        pickle.dump(reduce_reviews, output_file)

Overwriting dns_reviews.py


In [108]:
get_all_products(PAGES_LIST[0])

>>> Parsing page https://www.dns-shop.ru/catalog/17a8a01d16404e77/smartfony/?p=1


['https://www.dns-shop.ru/product/831d9dda31643330/4-smartfon-dexp-a140-8-gb-zolotistyj/',
 'https://www.dns-shop.ru/product/2813dfb931643330/4-smartfon-dexp-a140-8-gb-cernyj/',
 'https://www.dns-shop.ru/product/f2f4a71bff723332/4-smartfon-inoi-1-lite-4-gb-cernyj/',
 'https://www.dns-shop.ru/product/3ad8e2b7b8633330/45-smartfon-finepower-s5-4-gb-cernyj/',
 'https://www.dns-shop.ru/product/13a5e259fa663330/4-smartfon-finepower-c4-4-gb-cernyj/',
 'https://www.dns-shop.ru/product/5988bc619a0c3330/4-smartfon-micromax-q301-4-gb-cernyj/',
 'https://www.dns-shop.ru/product/63efa04201a93332/4-smartfon-micromax-q306-4-gb-cernyj/',
 'https://www.dns-shop.ru/product/84aa5d94456b477b/5-smartfon-finepower-c1-4-gb-zolotistyj/',
 'https://www.dns-shop.ru/product/5c99ef8f15e93330/5-smartfon-finepower-c1-4-gb-cernyj/',
 'https://www.dns-shop.ru/product/c4dd3dc652523330/5-smartfon-finepower-d3-4-gb-zolotistyj/',
 'https://www.dns-shop.ru/product/2f2e224c492b1b80/4-smartfon-bright--quick-bq-4001g-cool-8-

In [107]:
get_opinions('https://www.dns-shop.ru/product/831d9dda31643330/4-smartfon-dexp-a140-8-gb-zolotistyj/')

	>>> Parsing product https://www.dns-shop.ru/product/831d9dda31643330/4-smartfon-dexp-a140-8-gb-zolotistyj/


[('Цена. Процессор для такой цены относительно шустрый. Экран. Углы обзора и разрешение. Нет 4G, слабая батарея, нет подсветки кнопок, нет фонарика. Мало внутренней памяти, а карты памяти у нас на работе запрещены. Этот смарт на андроиде на сегодняшний день - единственное предложение в магазинах электроники, где нет камеры, для заводов, где это требуют. Хотелось бы, чтобы DNS это учел и выпустил ещё ряд более дорогих моделей без камеры.',
  5),
 ('Цена Сбрасывает вызов через минуту разговора Ниже я уже написал отзыв. Сейчас хочу дополнить. От данного аппарата требовалось выполнение самых простейших функций "звонилки", с которой он тоже не справляется. Больше минуты поговорить он не дает. Связь обрывает. Сброс ничего не дал. В настройках никаких лимитов ни на что не устанавливается. Не рекомендую.',
  1),
 ('- Дешёвый - без камеры - собран без всяких люфтов, скрипов и тд. - андройд, пусть и обрезанный - очень слабый. - подглючивает даже на стандартных операциях - плохой и малоемкий акку

In [2]:
with open('all_dns.txt', 'rb') as f:
    reviews = pickle.load(f)

In [4]:
len(reviews)

2904

In [5]:
reviews[0]

('батарея ,громкий кривая реализация с кнопками взял сегодня в белом цвете матери, мои впечатления.Скользкий,экран откровенно тухлый,установил яндекс браузер,скайп лайт,вайбер памяти без карточки осталось 32 метра,при этом не тупит,устанавливал арком,родной магазин вгружается ,быстро все это в нем есть,очень громкий,слышимость хорошая, отключаемая кнопка сос,2 лончера,один с плитками другой стандарт,очень мелкие виджеты,камера без фокуса програмно разогнана до 5 текст сильно мылит,батарейка по весу тянет на 2.500 расходуется равномерно,нет автоповорота,хоть галку поставил,при звонках экран тухнет и загорается,на экране сразу пленка,в скайпе громкий,есть 2 клавиатуры одна для кнопок работает криво,для андроида работает с экрана нормально,симкарты стандартные обе,мое впечатление,обычный смартфон 3,5 плюс клавиатура и неплохая батарея,',
 3)